In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

file_path = "/kaggle/input/global-air-pollution-dataset/global air pollution dataset.csv"
data = pd.read_csv(file_path)

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = data.dropna()
data = data.drop_duplicates()

# Розрахунок описової статистики
Обчислимо детальну описову статистику та визначимо будь-які викиди в наборі даних.

In [2]:
# Calculate descriptive statistics
descriptive_stats = data.describe()

# Identify outliers using the IQR method for each AQI Value column
def identify_outliers(column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Identify outliers for each AQI value
outliers_aqi = identify_outliers('AQI Value')
outliers_co = identify_outliers('CO AQI Value')
outliers_ozone = identify_outliers('Ozone AQI Value')
outliers_no2 = identify_outliers('NO2 AQI Value')
outliers_pm25 = identify_outliers('PM2.5 AQI Value')

descriptive_stats, outliers_aqi.shape[0], outliers_co.shape[0], outliers_ozone.shape[0], outliers_no2.shape[0], outliers_pm25.shape[0]

(          AQI Value  CO AQI Value  Ozone AQI Value  NO2 AQI Value  \
 count  23035.000000  23035.000000     23035.000000   23035.000000   
 mean      72.344693      1.376254        35.233905       3.084741   
 std       56.360992      1.844926        28.236613       5.281708   
 min        6.000000      0.000000         0.000000       0.000000   
 25%       39.000000      1.000000        21.000000       0.000000   
 50%       55.000000      1.000000        31.000000       1.000000   
 75%       80.000000      1.000000        40.000000       4.000000   
 max      500.000000    133.000000       235.000000      91.000000   
 
        PM2.5 AQI Value  
 count     23035.000000  
 mean         68.883482  
 std          55.057396  
 min           0.000000  
 25%          35.000000  
 50%          54.000000  
 75%          79.000000  
 max         500.000000  ,
 2875,
 8442,
 1500,
 1670,
 2628)

- Середнє значення AQI становить 72,34 із стандартним відхиленням 56,36. Мінімальне  значення AQI становить 6, а максимальне – 500, що свідчить про значну мінливість якості повітря.
- Значення CO AQI відносно низькі, із середнім значенням 1,38 і стандартним відхиленням 1,84. Значення в діапазоні від 0 до 133, вказуючи на те, що хоча більшість значень є низькими, є деякі екстремальні випадки.
- Значення AQI озону мають середнє значення 35,23 і стандартне відхилення 28,24, що свідчить про помірну мінливість.
- Значення NO2 AQI, як правило, низькі, із середнім значенням 3,08 і стандартним відхиленням 5,28.
- Значення AQI PM2,5 мають середнє значення 68,88 і стандартне відхилення 55,06, що вказує на значну варіабельність.
- У даних про значення індексу якості повітря (AQI) є 2875 викидів, що вказує на значну кількість екстремальних значень.
- Дані про значення CO AQI мають дуже велику кількість викидів, 8442, що свідчить про багато екстремальних значень у наборі даних.
- Для значень AQI озону існує 1500 викидів, що вказує на значну мінливість і екстремальні значення.
- Дані про значення NO2 AQI містять 1670 викидів, що вказує на наявність кількох екстремальних значень.
- Значення AQI PM2,5 мають 2628 викидів, що відображає значну мінливість і багато екстремальних значень у наборі даних.

# Створення математичної моделі
Створимо багатофакторну регресійну модель для аналізу зв’язку між значеннями AQI та значеннями AQI окремих забруднюючих речовин(CO, Ozone, $NO_2$, $PM_{2.5}$).

In [3]:
import statsmodels.api as sm

# Define the independent variables (predictors)
X = data[['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']]

# Define the dependent variable (response)
y = data['AQI Value']

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the model summary
model_summary = model.summary()

model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              AQI Value   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 2.228e+05
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        18:10:55   Log-Likelihood:                -83158.
No. Observations:               23035   AIC:                         1.663e+05
Df Residuals:                   23030   BIC:                         1.664e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6432      0.112     -5.721      0.000      -0.864      -0.423
CO AQI Value        0.0109      0.040      0.273      0.785      -0.067       0.089
Ozone AQI Value     0.1556      0.002     66.208      0.000       0.151       0.160
NO2 AQI Value      -0.0368      0.014     -2.714      0.007      -0.063      -0.010
PM2.5 AQI Value     0.9814      0.001    772.464      0.000       0.979       0.984
==============================================================================
Omnibus:                    22523.913   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1847377.782
Skew:                           4.633   Prob(JB):                         0.00
Kurtosis:                      45.883   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Регресійна модель
Далі створимо регресійну модель:

In [4]:
import statsmodels.api as sm

# Define the independent variables (predictors)
X = data[['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']]

# Define the dependent variable (response)
y = data['AQI Value']

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the model summary
model_summary = model.summary()
print(model_summary)

                            OLS Regression Results                            
Dep. Variable:              AQI Value   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 2.228e+05
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        18:10:55   Log-Likelihood:                -83158.
No. Observations:               23035   AIC:                         1.663e+05
Df Residuals:                   23030   BIC:                         1.664e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6432      0.112     

На основі регресійного аналізу бачимо, що значення AQI для озону та значення AQI PM2,5 є найбільш значущими предикторами загального індексу якості повітря, AQI, обидва демонструють сильний позитивний зв’язок із AQI. Значення AQI NO2 має незначний негативний вплив на AQI, тоді як значення AQI CO не впливає суттєво на AQI. Модель пояснює 97,5% дисперсії значень AQI, що вказує на її високу ефективність.